In [1]:
import pandas as pd
import numpy as np
import openpyxl as ox
import re
import editdistance
from datetime import datetime

In [2]:
df = pd.read_excel("columns.xlsx")

In [3]:
df

,Менеджер,Вид доставки,Дата,Товар,Производитель/Завод,Примечание к Товару,Кол-во,Ед.изм.,Машина,Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Примечание Иное


In [4]:
managers = pd.read_excel("dictionary.xlsx", sheet_name="managers")
products = pd.read_excel("dictionary.xlsx", sheet_name="products")

In [125]:
# managers = pd.DataFrame({'login': ['ИГО', 'Юра Менеджер', 'Алексей Мельхер'],
#                         'manager': ['Игорь Хабаров', 'Юрий', 'Алексей Мельхер']})

In [126]:
# products = pd.Series(data=['ЦЕМ I 42.5Н БЦК','Щебень гранитный 5-20 (ЛСР)','ЦЕМ I 42.5Н Костюковичи',
#                                'ЦЕМ I 42.5Н БЦЗ Костюковичи', 'ЦЕМ I 42.5Н', 'ПЕСКОГРУНТ'], name='product')

In [104]:
with pd.ExcelWriter("dictionary.xlsx") as writer:
    managers.to_excel(writer, sheet_name="managers", index=False)
    products.to_excel(writer, sheet_name="products", index=False)

In [57]:
products.str.replace(' ', '')

0                ЦЕМI42.5НБЦК
1    Щебеньгранитный5-20(ЛСР)
2        ЦЕМI42.5НКостюковичи
3     ЦЕМI42.5НБЦЗКостюковичи
4                   ЦЕМI42.5Н
5                  ПЕСКОГРУНТ
dtype: object

## Text

In [130]:
appl = """
Юра Менеджер:
1. Дата отгрузки:
26.04.2024                     
2. От ООО Спарта
3. Марка: Щебень гранитный 5-20(ЛСР)
Обязательно актуальный паспорт!
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество 120т
7. Машина: 
О327ВН790 МАN
"""

In [131]:
# Используем регулярное выражение для поиска даты в формате dd.mm.yyyy
date_pattern = r'\b\d{2}\.\d{2}(?:\.\d{2}(?:\d{2})?)?\b'

# Находим все совпадения с помощью метода findall
dates = re.findall(date_pattern, appl)

## Check

In [132]:
dates

['26.04.2024']

In [133]:
# функция для форматирования даты в установленный формат
def convert_to_full_year(date_str):
    current_year = datetime.now().year
    parts = date_str.split('.')
    
    # Пытаемся интерпретировать дату
    try:
        # Если в строке только две части, добавляем текущий год
        if len(parts) == 2:
            date_str += f".{current_year}"
        
        # Пробуем преобразовать строку в дату
        datetime_obj = datetime.strptime(date_str, '%d.%m.%Y')
    except ValueError:
        # Если возникает ошибка, попробуем считать год в формате yy
        try:
            datetime_obj = datetime.strptime(date_str, '%d.%m.%y')
        except ValueError:
            # Если и это не сработает, возможно, формат даты неизвестен
            raise ValueError("Неподдерживаемый формат даты")
    
    # Возвращаем дату в формате dd.mm.yyyy
    return datetime_obj.strftime('%d.%m.%Y')

In [134]:
# конвертируем ПОКА ЧТО для одной даты
convert_date = convert_to_full_year(dates[0])

## Check

In [135]:
convert_date

'26.04.2024'

In [136]:
def find_manager(text, managers_df):
    # Проходим по каждому логину менеджера
    for index, row in managers_df.iterrows():
        login = row['login']
        manager = row['manager']
        
        # Проверяем совпадение по колонке 'login'
        if login.lower().replace(' ', '') in text.lower().replace(' ', ''):
            return manager
    return np.NAN



In [137]:
manager_found = find_manager(appl, managers)

In [138]:
manager_found

'Юрий'

In [139]:
def delivery_type(text):
    if "самовывоз" in text.lower():
        return "самовывоз"
    return "доставка"

In [140]:
delivery_found = delivery_type(appl)

In [141]:
delivery_found

'доставка'

In [69]:
# def find_product(text, product_df):
#     """
#     Функция для поиска товара по тексту и датафрейму с видами товара.

#     Аргументы:
#     text (str): Текст для поиска товара.
#     product_df (pandas.DataFrame): Датафрейм с колонками 'login' и 'manager',
#                                      содержащий информацию о менеджерах.

#     Возвращает:
#     str: Имя менеджера, если совпадение найдено, иначе сообщение о том, что совпадений не найдено.
#     """
#     # Проходим по каждому логину менеджера
#     for index, row in product_df.iterrows():
#         product = row['product']
#         # Проверяем совпадение по колонке 'login'
#         if product.lower().replace(' ', '') in text.lower().replace(' ', ''):
#             return product
#     # Если совпадений не найдено
#     return np.NAN

In [70]:
# product_found = find_product(appl, products)

In [71]:
# product_found

'ЦЕМ I 42.5Н'

In [142]:
def find_product(text):
    # Находим значение после слова "Марка:" без учета регистра
    marca_match = re.search(r'Марка(:)?\s*(цемента)?\s*(.*?)\n', text, re.IGNORECASE)
    marca_value = marca_match.group(3)

    return marca_value
    
def find_product_notice(text):    
    # Находим значение между строкой с "Марка:" и следующим номером пункта
    # Если нет следующего номера пункта, возвращаем np.NAN
  
    intermediate_match = re.search(r'Марка.*?\n(.*?)\n\d', text, re.IGNORECASE | re.DOTALL)
    intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN
    
    if re.match(r'^[^0-9.].*', intermediate_value):
        return intermediate_value
    else:
        np.NAN

In [146]:
product_found = find_product(appl)

In [145]:
product_notice_found = find_product_notice(appl)

In [147]:
# Создаем новый DataFrame с одной строкой, содержащей только дату
new_row_df = pd.DataFrame({'Менеджер': [manager_found], 
                           'Вид доставки': [delivery_found], 
                           'Дата': [convert_date], 
                           'Товар': [product_found], 
#                            'Производитель/Завод',
                           'Примечание к Товару': [product_notice_found], 
#                            'Кол-во', 
#                            'Ед.изм.', 
#                            'Машина', 
#                            'Водитель', 
#                            'Продавец', 
#                            'Откуда', 
#                            'Покупатель', 
#                            'Грузополучатель ',
#                            'Адрес грузополучателя (юрид)', 
#                            'Адрес пункта разгрузки', 
#                            'Контакт гп',
#                            'Примечание Иное',
                          })

# Объединяем исходный DataFrame с новым DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

In [148]:
df

,Менеджер,Вид доставки,Дата,Товар,Производитель/Завод,Примечание к Товару,Кол-во,Ед.изм.,Машина,Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Примечание Иное
0,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),NaN,Обязательно актуальный паспорт!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df.drop(df.index, inplace=True)

In [ ]:
patterns = [
    # ПАО ГТС => ГТС, ПАО
    r'^(?i)(П?АО|ООО|ГУП)(?:\s+|\s*")([^"]+)"?',
    
    # ПАО Р-БАНК//ИЗМАЙЛОВ ВАЛЕРИЙ МИХАЙЛОВИЧ//РОССИЯ,МОСКВА Г Р/С в ПАО Р-БАНК г Москва => Измайлов В.
    r'//([\w-]+)\s+(\w)\w*\s+(?:\w)\w*//',
    
    # Индивидуальный ПРЕДПРИНИМАТЕЛЬ Логинов Андрей Николаевич => Логинов А., ИП
    r'^(?i)(и)(?:ндивидуальный)?\s*(п)(?:редприниматель)?\s*(\w+)\s+(\w)\w*\s*(\w)?\w*',
    
    # Никишкина Лидия Алексеевна (ИП) => Нокошина Л., ИП
    r'^([\w-]+)\s+(\w)\w*\s*(\w)?\w*\s*\((ИП)\)$',
    
    # ВОРОНОВ ИЛЬЯ АНДРЕЕВИЧ => Воронов И.
    r'([\w-]+)\s+(\w)\w*\s+(\w)\w*$',
    
    # ООО "ЧОП "ОХРАННЫЕ СИСТЕМЫ" => ОХРАННЫЕ СИСТЕМЫ, ЧОП, ООО
    r'^([\w-]{2,3})\s+"?(.*)\s+"(.*)"',
    
    #  contains "ФНС": 
    r'\w*\s*(?i)(?:и?)\s*(ф)(?:едеральной)?\s*(н)(?:алоговой)?\s*(с)(?:лужбы)?\s*(?:россии)?\s*\w*\s*', 
    
    # ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛАГ1" => ЛАГ1, ООО
    r'(\w)\w{3,}|\"([^\"]+)\"',
    
    # Межрегиональное операционное УФК (КАЗНАЧЕЙСТВО РОССИИ) => Межрегиональное операционное УФК
    r'^.+?(?=\s*\()',
]       

def extract_text(row):
    """
    Get string with raw title, extract necessary text by patterns and returns formatted values
    """
    matches = []

    for pattern in patterns:
        match = re.findall(pattern, row)
        matches.append(match)
    
    return '{}, {}, {}'.format(matches[5][0][2], matches[5][0][1], matches[5][0][0]) if matches[5] else \
        '{} {}.'.format(matches[1][0][0].capitalize(), matches[1][0][1]) if matches[1] else \
        '{}, {}'.format(matches[0][0][1], matches[0][0][0]) if matches[0] else \
        '{} {}.{}., {}{}'.format(matches[2][0][2].capitalize(), matches[2][0][3], matches[2][0][4], matches[2][0][0], matches[2][0][1]) if matches[2] else \
        '{} {}.{}., {}'.format(matches[3][0][0].capitalize(), matches[3][0][1], matches[3][0][2], matches[3][0][3]) if matches[3] else \
        '{}{}{}'.format(matches[6][0][0].upper(),matches[6][0][1].upper(),matches[6][0][2]).upper() if matches[6] else \
        '{} {}.{}.'.format(matches[4][0][0].capitalize(), matches[4][0][1], matches[4][0][2]) if matches[4] else \
        '{}, {}'.format([mat[1] for mat in matches[7] if mat[1]][0],
                        ''.join(map(str, [mat[0] for mat in matches[7] if
                                          mat[0]]))) if '"' in row else \
        '{}'.format(matches[8][0]) if matches[8] else \
        row

# creating column "Agent"
df['Agent'] = np.vectorize(extract_text)(df['Full_Name'])
df['Name'] = df['Agent'].map(lambda x: x.lower())

In [95]:
def fix_typo(original, target):
    distance = editdistance.eval(original, target)
    if distance <= 3:  # Пороговое значение для определения опечатки
        return target
    else:
        return original

original = "Щебин ранитный"
target = "Щебень гранитный"
fixed_string = fix_typo(original, target)
print("Fixed string:", fixed_string)

Fixed string: Щебень гранитный


In [57]:
import pandas as pd
import editdistance

# Заданный датафрейм managers
managers = pd.DataFrame({'login': ['ИГО', 'Юра Менеджер', 'Алексей Мельхер'],
                        'manager': ['Игорь Хабаров', 'Юрий', 'Алексей Мельхер']})

# Текст для проверки
text = "Алексей Мельхер: Заявка на доставку 1.Дата отгрузки 28.04 2. Марка: ЦЕМ I 42.5Н Костюковичи 3. Количество: 70 тонн"

# Функция для нахождения наилучшего соответствия менеджера в тексте с учетом опечаток
def find_best_match(text, managers):
    best_match = None
    min_distance = float('inf')  # Начальное значение для минимального расстояния
    
    for manager in managers['login']:
        distance = editdistance.eval(text.lower(), manager.lower())  # Сравниваем строки, приведенные к нижнему регистру
        if distance < min_distance:
            min_distance = distance
            best_match = manager
    
    if min_distance <= 2:  # Пороговое значение для определения опечаток
        return managers[managers['login'] == best_match]['manager'].iloc[0]
    else:
        return None

# Находим наилучшее соответствие менеджера в тексте
best_manager = find_best_match(text, managers)
if best_manager:
    print("Лучший соответствующий менеджер:", best_manager)
else:
    print("Менеджер не найден")


Менеджер не найден


In [139]:
import re
import numpy as np

text = """
1. Дата отгрузки: 28.04.24 
2. Марка Цем I 42.5Н Кричев
3. Количество: 40т.
"""

# Находим значение после слова "Марка:" без учета регистра
marca_match = re.search(r'Марка\s*(цемента)?\s*(.*?)\n', text, re.IGNORECASE)
marca_value = marca_match.group(2)

# Находим значение между строкой с "Марка:" и следующим номером пункта
# Если нет следующего номера пункта, возвращаем np.NAN
intermediate_match = re.search(r'Марка.*?\n(.*?)\n\d', text, re.IGNORECASE | re.DOTALL)
intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN

print("Значение после слова 'Марка:':", marca_value)
print("Значение между строкой с 'Марка:' и следующим номером:", intermediate_value)


Значение после слова 'Марка:': Цем I 42.5Н Кричев
Значение между строкой с 'Марка:' и следующим номером: nan


In [157]:
import re
import numpy as np

text = """
1. Дата отгрузки: 28.04.24 
2. Марка Цем I 42.5Н Кричев
3. Количество: 40т.
"""

# Находим значение после слова "Марка:" без учета регистра
marca_match = re.search(r'Марка\s*(цемента)?\s*(.*?)\n', text, re.IGNORECASE)
marca_value = marca_match.group(2)

# Находим значение между строкой с "Марка:" и следующим номером пункта
# Если нет следующего номера пункта, возвращаем np.NAN
intermediate_match = re.search(r'Марка.*?\n(.*?)\n\d', text, re.IGNORECASE | re.DOTALL)
intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN

print("Значение после слова 'Марка:':", marca_value)
print("Значение между строкой с 'Марка:' и следующим номером:", intermediate_value)


Значение после слова 'Марка:': Цем I 42.5Н Кричев
Значение между строкой с 'Марка:' и следующим номером: nan


In [64]:
pattern = r'\d+\..*'
text = "5. Надо полис"

match = re.match( r'\d+\..*', text)

In [65]:
match[0]

'5. Надо полис'

In [83]:
type(re.match(r'^[^0-9.].*', "5. Надо полис"))

NoneType